**3 Steps**

This is so easy from hugging face we use transformers
1.  In transformers we have two parameter
*   Tokenizer
*   Model

2.  Tokenizer encode our sequences

3.   Models takes encoded data as input and gives output in form of value from 1 to 5





**Install and import dependencies**

In [49]:
from transformers import Pipeline
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import tensorflow as tf
import numpy as np
import torch #to use argmax we are using pytorch

In [6]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

**Initializing model**


In [9]:
modelname="nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer=AutoTokenizer.from_pretrained(modelname)
model=AutoModelForSequenceClassification.from_pretrained(modelname)


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [54]:
tokens=tokenizer.encode('i is not good',return_tensors='pt')

In [55]:
token=tokenizer.decode(tokens[0]) ##,skip_special_tokens=True to remove tokens
token

'[CLS] i is not good [SEP]'

In [56]:
res=model(tokens)
res

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.8348,  2.5366,  1.0322, -2.0771, -3.4954]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [57]:
res.logits[0]

tensor([ 2.8348,  2.5366,  1.0322, -2.0771, -3.4954],
       grad_fn=<SelectBackward0>)

In [58]:
int(torch.argmax(res.logits[0]))+1

1

Re-writing the same code in a function for reuse

In [59]:
def sentiment_text(sequence):
  tokens=tokenizer.encode(sequence,return_tensors='pt')
  sent=model(tokens)
  val=torch.argmax(sent.logits[0])
  return int(val)+1

**Collecting food review data from yelp.com website**

In [69]:
req=requests.get("https://www.yelp.com/biz/taco-bell-san-francisco-12")
soup=BeautifulSoup(req.text,'html.parser')
ret=re.compile('.*comment.*')
results=soup.find_all('p',{'class':ret})
reviews=[result.text for result in results]

In [71]:
reviews[0]

"I enjoyed Taco Bell like it was an expensive fancy meal lol not. I went in at this location at around to try the new chile verde fries and an enchirito.  The chile verde fries were as fresh as they could get. The food was even still steaming from how hot it was. I can't remember the last time I enjoyed Taco Bell like this. The neighborhood is great, and the employees working at this location were far more mature than those of other locations I have visited in the past.  Definitely my go to place in a pinch."

In [79]:
df=pd.DataFrame(reviews,columns=['Review'])

In [80]:
df.head()

,Review
0,I enjoyed Taco Bell like it was an expensive f...
1,"""Taco Bell has always been my go-to spot for a..."
2,I hate this place. I ate their tacos for lunch...
3,I was here again about 3wks ago and I'm 1000% ...
4,Taco Bells and McDonalds..hahah KFC fried chic...


In [92]:
df['Ratings']=df['Review'].apply(sentiment_text)

In [93]:
df

,Review,Ratings
0,I enjoyed Taco Bell like it was an expensive f...,4
1,"""Taco Bell has always been my go-to spot for a...",5
2,I hate this place. I ate their tacos for lunch...,1
3,I was here again about 3wks ago and I'm 1000% ...,5
4,Taco Bells and McDonalds..hahah KFC fried chic...,3
5,I asked for a Baja blast and she gave me boba...,1
6,Yum ba hahah blastie poo,5
7,I love the new cheeze it Crunchwrap! So yummy!...,5
8,"Tasty food, quick service. It's my goto place ...",4
9,mario and the manager hunter were so amazing!!...,5


**Writing dataframe to excel file**

In [97]:
df.to_csv('Restaurant review.csv',index=False)